In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from time import time
from tqdm import tqdm_notebook as tqdm
from collections import Counter
from scipy import stats
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold, StratifiedKFold
import gc
import json
pd.set_option('display.max_columns', 1000)

from pathlib import Path

In [2]:
path=Path('/kaggle/data_science_bowl')
path

PosixPath('/kaggle/data_science_bowl')

In [3]:
def read_data():
    train_df = pd.read_csv(path/'train.csv')
    test_df = pd.read_csv(path/'test.csv')
    train_labels_df = pd.read_csv(path/'train_labels.csv')
    specs_df = pd.read_csv(path/'specs.csv')
    return train_df, test_df, train_labels_df, specs_df

In [4]:
train_df, test_df, train_labels_df, specs_df = read_data()

In [5]:
train_df.columns

Index(['event_id', 'game_session', 'timestamp', 'event_data',
       'installation_id', 'event_count', 'event_code', 'game_time', 'title',
       'type', 'world'],
      dtype='object')

## Feature Engineering

In [6]:
def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    # convert text into datetime
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code

In [7]:
train_df, test_df, train_labels_df, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(train_df, test_df, train_labels_df)

In [8]:
# this is the function that convert the raw data into processed features
def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    # Constants and parameters declaration
    last_activity = 0
    
    user_activities_count = {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
    
    # new features: time spent in each activity
    last_session_time_sec = 0
    accuracy_groups = {0:0, 1:0, 2:0, 3:0}
    all_assessments = []
    accumulated_accuracy_group = 0
    accumulated_accuracy = 0
    accumulated_correct_attempts = 0 
    accumulated_uncorrect_attempts = 0
    accumulated_actions = 0
    counter = 0
    time_first_activity = float(user_sample['timestamp'].values[0])
    durations = []
    last_accuracy_title = {'acc_' + title: -1 for title in assess_titles}
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
    
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
                    
            
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features = user_activities_count.copy()
            features.update(last_accuracy_title.copy())
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features.update(last_accuracy_title.copy())
            
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            features['accumulated_correct_attempts'] = accumulated_correct_attempts
            features['accumulated_uncorrect_attempts'] = accumulated_uncorrect_attempts
            accumulated_correct_attempts += true_attempts 
            accumulated_uncorrect_attempts += false_attempts
            # the time spent in the app so far
            if durations == []:
                features['duration_mean'] = 0
            else:
                features['duration_mean'] = np.mean(durations)
            durations.append((session.iloc[-1, 2] - session.iloc[0, 2] ).seconds)
            # the accurace is the all time wins divided by the all time attempts
            features['accumulated_accuracy'] = accumulated_accuracy/counter if counter > 0 else 0
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            accumulated_accuracy += accuracy
            last_accuracy_title['acc_' + session_title_text] = accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            features.update(accuracy_groups)
            accuracy_groups[features['accuracy_group']] += 1
            # mean of the all accuracy groups of this player
            features['accumulated_accuracy_group'] = accumulated_accuracy_group/counter if counter > 0 else 0
            accumulated_accuracy_group += features['accuracy_group']
            # how many actions the player has done so far, it is initialized as 0 and updated some lines below
            features['accumulated_actions'] = accumulated_actions
            
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                all_assessments.append(features)
                
            counter += 1
        
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')

        # counts how many actions the player has done so far, used in the feature of the same name
        accumulated_actions += len(session)
        if last_activity != session_type:
            user_activities_count[session_type] += 1
            last_activitiy = session_type 
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments

In [9]:
compiled_train = []
for i, (ins_id, user_sample) in tqdm(enumerate(train_df.groupby('installation_id', sort = False)), total = 17000):
    compiled_train += get_data(user_sample)

In [11]:
compiled_test = []
for ins_id, user_sample in tqdm(test_df.groupby('installation_id', sort = False), total = 1000):
        test_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)

In [46]:
comp_train_df = pd.DataFrame(compiled_train)
comp_train_df

,Clip,Activity,Assessment,Game,acc_Mushroom Sorter (Assessment),acc_Cart Balancer (Assessment),acc_Chest Sorter (Assessment),acc_Bird Measurer (Assessment),acc_Cauldron Filler (Assessment),2050,4100,2060,4110,2070,2075,2080,2081,2083,3110,3120,3121,4220,4230,5000,4235,5010,4010,4020,4021,4022,4025,4030,4031,3010,4035,4040,3020,3021,4045,2000,4050,2010,2020,4070,2025,2030,4080,2035,2040,4090,4095,5c2f29ca,6bf9e3e1,99ea62f3,84b0e0c8,907a054b,71e712d8,67439901,cb1178ad,1340b8d7,7d5c30a2,eb2c19cd,6f4adc4b,c58186bf,a7640a16,b1d5101d,13f56524,69fdac0a,a6d66e51,e57dd7af,45d01abe,cc5087a3,cf7638f3,ab4ec3a4,857f21c0,763fc34e,4a09ace1,73757a5e,7423acbc,5dc079d8,53c6e11a,598f4598,29f54413,04df9b66,08ff79ad,5e3ea25a,e79f3763,5290eab1,f7e47413,e3ff61fb,71fe8f75,bdf49a58,3d63345e,2c4e6db0,dcaede90,e720d930,3393b68b,9b23e8ee,a1bbe385,c2baf0bd,cb6010f8,7961e599,d122731b,2dc29e21,222660ff,363d3849,db02c830,d38c2fd7,731c0cbe,15ba1109,58a0de5c,5f5b2617,f71c4741,363c86c9,ac92046e,19967db1,5859dfb6,5e109ec3,26fd2d99,02a42007,daac11b0,63f13dd7,b7530680,99abe2bb,9e34ea74,3bb91dda,51102b85,df4940d3,88d4a5be,49ed92e9,06372577,9ee1c98c,36fa3ebe,bcceccc6,9b4001e4,155f62a4,0330ab6a,1cf54632,7372e1a5,b88f38da,c74f40cd,9ed8f6da,0d1da71f,fd20ea40,bc8f2793,d45ed6a1,bd701df8,895865f3,5a848010,a1192f43,a16a373e,2dcad279,26a5a3dd,d06f75b5,30df3273,160654fd,6cf7d25c,ca11f653,beb0a7b9,28520915,4e5fc6f5,736f9581,b7dc8128,d02b7a8e,29a42aea,56817e2b,1beb320a,562cec5f,756e5507,3dcdda7f,9c5ef70c,dcb55a27,ecc36b7f,2a444e03,93edfe2e,acf5c23f,90ea0bac,d9c005dd,30614231,f50fc6c1,9d4e7b25,6c930e6e,a1e4395d,ad2fc29c,00c73085,77261ab5,37ee8496,e7e44842,15a43e5b,461eace6,77ead60d,4074bac2,55115cbd,8d748b58,cfbd47c8,16667cc5,6077cc36,2b9272f4,47f43a44,15f99afc,d51b1749,2b058fe3,5348fd84,b012cd7f,65abac75,070a5291,e64e2cfd,832735e1,c1cac9a2,4901243f,3d8c61b0,c6971acf,7f0836bf,a29c5338,c0415e5c,3d0b9317,d88ca108,e080a381,5de79a6a,923afab1,7dfe6d8a,1f19558b,4b5efe37,1575e76c,83c6c409,709b1251,2ec694de,7ec0c298,3babcb9b,7525289a,1cc7cfca,3bf1cf26,46b50ba8,e04fb33d,fcfdffb6,85de926c,ecaab346,e5734469,0d18d96c,56cd3b43,df4fe8b6,9b01374f,d88e8f25,0a08139c,499edb7c,44cb4907,2230fab4,15eb4a7d,7cf1bc53,29bdd9ba,51311d7a,4ef8cdd3,f5b8c21a,ad148f58,a8876db3,f6947f54,b80e5e84,7fd1ac25,4d6737eb,3bb91ced,7ad3efc6,dcb1663e,4a4c3d21,565a3990,8f094001,a5e9da97,ec138c1c,a0faea5d,c277e121,31973d56,b74258a0,9de5e594,a8efe47b,d3f1e122,0413e89d,de26c3a6,e694a35b,5e812b27,c7128948,d2e9262e,7d093bf9,7040c096,28a4eb9a,01ca3a3c,cdd22e43,bb3e370b,b120f2ac,792530f8,1996c610,77c76bc5,6f445b57,1375ccb7,8d84fa81,f54238ee,3ccd3f02,5b49460a,804ee27f,c54cf6c5,828e68f9,65a38bf7,ecc6157f,0086365d,4d911100,67aa2ada,a76029ee,9ce586dd,c952eb01,84538528,532a2afb,262136f4,392e14df,c7fe2a55,6c517a88,2fb91ec1,bbfe0445,022b4259,3afb49e6,1b54d27f,93b353f2,3323d7e9,16dffff1,37c53127,90d848e0,3a4be871,74e5f8a7,e7561dd2,f32856e4,6aeafed4,33505eae,611485c5,1bb5fbdb,6f8106d9,91561152,3ee399c3,3dfd4aa4,5d042115,76babcde,3ddc79c3,5154fc30,3bfd1a65,8af75982,6f4bd64e,b2dba42b,250513af,05ad839b,ea296733,f93fc684,f56e0afc,6088b756,5f0eb72c,8ac7cce4,a2df0760,4c2ec19f,e4d32835,119b5b02,9e6b7fb5,4bb2f698,0db6d71d,6043a2b4,003cd2ee,f3cd5473,a8a78786,85d1b0de,b5053438,a5be6304,08fd73f3,a52b92d5,6d90d394,86ba578b,d2659ab4,1c178d24,17ca3959,17113b36,86c924c4,587b5989,9e4c8c7b,fbaf3456,25fa8af4,9d29771f,f28c589a,884228c8,8b757ab8,e4f1efe6,ea321fb1,bfc77bd6,87d743c1,c7f7f0e1,e37a2b78,a8cc6fec,795e4a37,14de4c5d,2a512369,47efca07,37db1c2f,b2e5b0f1,c51d8688,92687c59,7da34a02,d185d3ea,5c3d2b2f,a44b10dc,47026d5f,37937459,d3640339,f806dc10,9554a50b,5be391b5,38074c54,56bcd38d,0ce40006,e9c52111,d3268efa,abc5811c,3afde5dd,3b2048ee,ab3136ba,cf82af56,28ed704e,8d7e386c,d2278a3b,28f975ea,90efca10,e5c9df6f,c189aaf2,bd612267,7ab78247,27253bdc,89aace00,8fee50e2,1af8be29,a592d54e,46cd75b4,1325467d,48349b14,3edf6747,Mushroom Sorter (Assessment),Ordering Spheres,Chest Sorter (Assessment),Happy Camel,Chicken Balancer (Activity),Rulers,Crystals Rule,Dino Dive,Di

In [45]:
comp_train_df['session_title'].unique()

array([ 0, 24, 33, 35,  2])

In [47]:
comp_test_df = pd.DataFrame(compiled_test)
comp_test_df.shape

(1000, 895)

In [15]:
comp_test_df

,Clip,Activity,Assessment,Game,acc_Mushroom Sorter (Assessment),acc_Cart Balancer (Assessment),acc_Chest Sorter (Assessment),acc_Bird Measurer (Assessment),acc_Cauldron Filler (Assessment),2050,4100,2060,4110,2070,2075,2080,2081,2083,3110,3120,3121,4220,4230,5000,4235,5010,4010,4020,4021,4022,4025,4030,4031,3010,4035,4040,3020,3021,4045,2000,4050,2010,2020,4070,2025,2030,4080,2035,2040,4090,4095,5c2f29ca,6bf9e3e1,99ea62f3,84b0e0c8,907a054b,71e712d8,67439901,cb1178ad,1340b8d7,7d5c30a2,eb2c19cd,6f4adc4b,c58186bf,a7640a16,b1d5101d,13f56524,69fdac0a,a6d66e51,e57dd7af,45d01abe,cc5087a3,cf7638f3,ab4ec3a4,857f21c0,763fc34e,4a09ace1,73757a5e,7423acbc,5dc079d8,53c6e11a,598f4598,29f54413,04df9b66,08ff79ad,5e3ea25a,e79f3763,5290eab1,f7e47413,e3ff61fb,71fe8f75,bdf49a58,3d63345e,2c4e6db0,dcaede90,e720d930,3393b68b,9b23e8ee,a1bbe385,c2baf0bd,cb6010f8,7961e599,d122731b,2dc29e21,222660ff,363d3849,db02c830,d38c2fd7,731c0cbe,15ba1109,58a0de5c,5f5b2617,f71c4741,363c86c9,ac92046e,19967db1,5859dfb6,5e109ec3,26fd2d99,02a42007,daac11b0,63f13dd7,b7530680,99abe2bb,9e34ea74,3bb91dda,51102b85,df4940d3,88d4a5be,49ed92e9,06372577,9ee1c98c,36fa3ebe,bcceccc6,9b4001e4,155f62a4,0330ab6a,1cf54632,7372e1a5,b88f38da,c74f40cd,9ed8f6da,0d1da71f,fd20ea40,bc8f2793,d45ed6a1,bd701df8,895865f3,5a848010,a1192f43,a16a373e,2dcad279,26a5a3dd,d06f75b5,30df3273,160654fd,6cf7d25c,ca11f653,beb0a7b9,28520915,4e5fc6f5,736f9581,b7dc8128,d02b7a8e,29a42aea,56817e2b,1beb320a,562cec5f,756e5507,3dcdda7f,9c5ef70c,dcb55a27,ecc36b7f,2a444e03,93edfe2e,acf5c23f,90ea0bac,d9c005dd,30614231,f50fc6c1,9d4e7b25,6c930e6e,a1e4395d,ad2fc29c,00c73085,77261ab5,37ee8496,e7e44842,15a43e5b,461eace6,77ead60d,4074bac2,55115cbd,8d748b58,cfbd47c8,16667cc5,6077cc36,2b9272f4,47f43a44,15f99afc,d51b1749,2b058fe3,5348fd84,b012cd7f,65abac75,070a5291,e64e2cfd,832735e1,c1cac9a2,4901243f,3d8c61b0,c6971acf,7f0836bf,a29c5338,c0415e5c,3d0b9317,d88ca108,e080a381,5de79a6a,923afab1,7dfe6d8a,1f19558b,4b5efe37,1575e76c,83c6c409,709b1251,2ec694de,7ec0c298,3babcb9b,7525289a,1cc7cfca,3bf1cf26,46b50ba8,e04fb33d,fcfdffb6,85de926c,ecaab346,e5734469,0d18d96c,56cd3b43,df4fe8b6,9b01374f,d88e8f25,0a08139c,499edb7c,44cb4907,2230fab4,15eb4a7d,7cf1bc53,29bdd9ba,51311d7a,4ef8cdd3,f5b8c21a,ad148f58,a8876db3,f6947f54,b80e5e84,7fd1ac25,4d6737eb,3bb91ced,7ad3efc6,dcb1663e,4a4c3d21,565a3990,8f094001,a5e9da97,ec138c1c,a0faea5d,c277e121,31973d56,b74258a0,9de5e594,a8efe47b,d3f1e122,0413e89d,de26c3a6,e694a35b,5e812b27,c7128948,d2e9262e,7d093bf9,7040c096,28a4eb9a,01ca3a3c,cdd22e43,bb3e370b,b120f2ac,792530f8,1996c610,77c76bc5,6f445b57,1375ccb7,8d84fa81,f54238ee,3ccd3f02,5b49460a,804ee27f,c54cf6c5,828e68f9,65a38bf7,ecc6157f,0086365d,4d911100,67aa2ada,a76029ee,9ce586dd,c952eb01,84538528,532a2afb,262136f4,392e14df,c7fe2a55,6c517a88,2fb91ec1,bbfe0445,022b4259,3afb49e6,1b54d27f,93b353f2,3323d7e9,16dffff1,37c53127,90d848e0,3a4be871,74e5f8a7,e7561dd2,f32856e4,6aeafed4,33505eae,611485c5,1bb5fbdb,6f8106d9,91561152,3ee399c3,3dfd4aa4,5d042115,76babcde,3ddc79c3,5154fc30,3bfd1a65,8af75982,6f4bd64e,b2dba42b,250513af,05ad839b,ea296733,f93fc684,f56e0afc,6088b756,5f0eb72c,8ac7cce4,a2df0760,4c2ec19f,e4d32835,119b5b02,9e6b7fb5,4bb2f698,0db6d71d,6043a2b4,003cd2ee,f3cd5473,a8a78786,85d1b0de,b5053438,a5be6304,08fd73f3,a52b92d5,6d90d394,86ba578b,d2659ab4,1c178d24,17ca3959,17113b36,86c924c4,587b5989,9e4c8c7b,fbaf3456,25fa8af4,9d29771f,f28c589a,884228c8,8b757ab8,e4f1efe6,ea321fb1,bfc77bd6,87d743c1,c7f7f0e1,e37a2b78,a8cc6fec,795e4a37,14de4c5d,2a512369,47efca07,37db1c2f,b2e5b0f1,c51d8688,92687c59,7da34a02,d185d3ea,5c3d2b2f,a44b10dc,47026d5f,37937459,d3640339,f806dc10,9554a50b,5be391b5,38074c54,56bcd38d,0ce40006,e9c52111,d3268efa,abc5811c,3afde5dd,3b2048ee,ab3136ba,cf82af56,28ed704e,8d7e386c,d2278a3b,28f975ea,90efca10,e5c9df6f,c189aaf2,bd612267,7ab78247,27253bdc,89aace00,8fee50e2,1af8be29,a592d54e,46cd75b4,1325467d,48349b14,3edf6747,Mushroom Sorter (Assessment),Ordering Spheres,Chest Sorter (Assessment),Happy Camel,Chicken Balancer (Activity),Rulers,Crystals Rule,Dino Dive,Di

In [21]:
comp_test_df[(comp_test_df['installation_id'] == '00abaee7')]

,Clip,Activity,Assessment,Game,acc_Mushroom Sorter (Assessment),acc_Cart Balancer (Assessment),acc_Chest Sorter (Assessment),acc_Bird Measurer (Assessment),acc_Cauldron Filler (Assessment),2050,4100,2060,4110,2070,2075,2080,2081,2083,3110,3120,3121,4220,4230,5000,4235,5010,4010,4020,4021,4022,4025,4030,4031,3010,4035,4040,3020,3021,4045,2000,4050,2010,2020,4070,2025,2030,4080,2035,2040,4090,4095,5c2f29ca,6bf9e3e1,99ea62f3,84b0e0c8,907a054b,71e712d8,67439901,cb1178ad,1340b8d7,7d5c30a2,eb2c19cd,6f4adc4b,c58186bf,a7640a16,b1d5101d,13f56524,69fdac0a,a6d66e51,e57dd7af,45d01abe,cc5087a3,cf7638f3,ab4ec3a4,857f21c0,763fc34e,4a09ace1,73757a5e,7423acbc,5dc079d8,53c6e11a,598f4598,29f54413,04df9b66,08ff79ad,5e3ea25a,e79f3763,5290eab1,f7e47413,e3ff61fb,71fe8f75,bdf49a58,3d63345e,2c4e6db0,dcaede90,e720d930,3393b68b,9b23e8ee,a1bbe385,c2baf0bd,cb6010f8,7961e599,d122731b,2dc29e21,222660ff,363d3849,db02c830,d38c2fd7,731c0cbe,15ba1109,58a0de5c,5f5b2617,f71c4741,363c86c9,ac92046e,19967db1,5859dfb6,5e109ec3,26fd2d99,02a42007,daac11b0,63f13dd7,b7530680,99abe2bb,9e34ea74,3bb91dda,51102b85,df4940d3,88d4a5be,49ed92e9,06372577,9ee1c98c,36fa3ebe,bcceccc6,9b4001e4,155f62a4,0330ab6a,1cf54632,7372e1a5,b88f38da,c74f40cd,9ed8f6da,0d1da71f,fd20ea40,bc8f2793,d45ed6a1,bd701df8,895865f3,5a848010,a1192f43,a16a373e,2dcad279,26a5a3dd,d06f75b5,30df3273,160654fd,6cf7d25c,ca11f653,beb0a7b9,28520915,4e5fc6f5,736f9581,b7dc8128,d02b7a8e,29a42aea,56817e2b,1beb320a,562cec5f,756e5507,3dcdda7f,9c5ef70c,dcb55a27,ecc36b7f,2a444e03,93edfe2e,acf5c23f,90ea0bac,d9c005dd,30614231,f50fc6c1,9d4e7b25,6c930e6e,a1e4395d,ad2fc29c,00c73085,77261ab5,37ee8496,e7e44842,15a43e5b,461eace6,77ead60d,4074bac2,55115cbd,8d748b58,cfbd47c8,16667cc5,6077cc36,2b9272f4,47f43a44,15f99afc,d51b1749,2b058fe3,5348fd84,b012cd7f,65abac75,070a5291,e64e2cfd,832735e1,c1cac9a2,4901243f,3d8c61b0,c6971acf,7f0836bf,a29c5338,c0415e5c,3d0b9317,d88ca108,e080a381,5de79a6a,923afab1,7dfe6d8a,1f19558b,4b5efe37,1575e76c,83c6c409,709b1251,2ec694de,7ec0c298,3babcb9b,7525289a,1cc7cfca,3bf1cf26,46b50ba8,e04fb33d,fcfdffb6,85de926c,ecaab346,e5734469,0d18d96c,56cd3b43,df4fe8b6,9b01374f,d88e8f25,0a08139c,499edb7c,44cb4907,2230fab4,15eb4a7d,7cf1bc53,29bdd9ba,51311d7a,4ef8cdd3,f5b8c21a,ad148f58,a8876db3,f6947f54,b80e5e84,7fd1ac25,4d6737eb,3bb91ced,7ad3efc6,dcb1663e,4a4c3d21,565a3990,8f094001,a5e9da97,ec138c1c,a0faea5d,c277e121,31973d56,b74258a0,9de5e594,a8efe47b,d3f1e122,0413e89d,de26c3a6,e694a35b,5e812b27,c7128948,d2e9262e,7d093bf9,7040c096,28a4eb9a,01ca3a3c,cdd22e43,bb3e370b,b120f2ac,792530f8,1996c610,77c76bc5,6f445b57,1375ccb7,8d84fa81,f54238ee,3ccd3f02,5b49460a,804ee27f,c54cf6c5,828e68f9,65a38bf7,ecc6157f,0086365d,4d911100,67aa2ada,a76029ee,9ce586dd,c952eb01,84538528,532a2afb,262136f4,392e14df,c7fe2a55,6c517a88,2fb91ec1,bbfe0445,022b4259,3afb49e6,1b54d27f,93b353f2,3323d7e9,16dffff1,37c53127,90d848e0,3a4be871,74e5f8a7,e7561dd2,f32856e4,6aeafed4,33505eae,611485c5,1bb5fbdb,6f8106d9,91561152,3ee399c3,3dfd4aa4,5d042115,76babcde,3ddc79c3,5154fc30,3bfd1a65,8af75982,6f4bd64e,b2dba42b,250513af,05ad839b,ea296733,f93fc684,f56e0afc,6088b756,5f0eb72c,8ac7cce4,a2df0760,4c2ec19f,e4d32835,119b5b02,9e6b7fb5,4bb2f698,0db6d71d,6043a2b4,003cd2ee,f3cd5473,a8a78786,85d1b0de,b5053438,a5be6304,08fd73f3,a52b92d5,6d90d394,86ba578b,d2659ab4,1c178d24,17ca3959,17113b36,86c924c4,587b5989,9e4c8c7b,fbaf3456,25fa8af4,9d29771f,f28c589a,884228c8,8b757ab8,e4f1efe6,ea321fb1,bfc77bd6,87d743c1,c7f7f0e1,e37a2b78,a8cc6fec,795e4a37,14de4c5d,2a512369,47efca07,37db1c2f,b2e5b0f1,c51d8688,92687c59,7da34a02,d185d3ea,5c3d2b2f,a44b10dc,47026d5f,37937459,d3640339,f806dc10,9554a50b,5be391b5,38074c54,56bcd38d,0ce40006,e9c52111,d3268efa,abc5811c,3afde5dd,3b2048ee,ab3136ba,cf82af56,28ed704e,8d7e386c,d2278a3b,28f975ea,90efca10,e5c9df6f,c189aaf2,bd612267,7ab78247,27253bdc,89aace00,8fee50e2,1af8be29,a592d54e,46cd75b4,1325467d,48349b14,3edf6747,Mushroom Sorter (Assessment),Ordering Spheres,Chest Sorter (Assessment),Happy Camel,Chicken Balancer (Activity),Rulers,Crystals Rule,Dino Dive,Di

In [24]:
features = comp_train_df.loc[(comp_train_df.sum(axis=1) != 0), (comp_train_df.sum(axis=0) != 0)].columns # delete useless columns
features = [x for x in features if x not in ['accuracy_group', 'installation_id']]
comp_train_df[features]

,Clip,Activity,Assessment,Game,acc_Mushroom Sorter (Assessment),acc_Cart Balancer (Assessment),acc_Chest Sorter (Assessment),acc_Bird Measurer (Assessment),acc_Cauldron Filler (Assessment),2050,4100,2060,4110,2070,2075,2080,2081,2083,3110,3120,3121,4220,4230,5000,4235,5010,4010,4020,4021,4022,4025,4030,4031,3010,4035,4040,3020,3021,4045,2000,4050,2010,2020,4070,2025,2030,4080,2035,2040,4090,4095,5c2f29ca,6bf9e3e1,99ea62f3,84b0e0c8,907a054b,71e712d8,67439901,cb1178ad,1340b8d7,7d5c30a2,eb2c19cd,6f4adc4b,c58186bf,a7640a16,b1d5101d,13f56524,69fdac0a,a6d66e51,e57dd7af,45d01abe,cc5087a3,cf7638f3,ab4ec3a4,857f21c0,763fc34e,4a09ace1,73757a5e,7423acbc,53c6e11a,598f4598,29f54413,04df9b66,08ff79ad,5e3ea25a,e79f3763,5290eab1,f7e47413,e3ff61fb,71fe8f75,bdf49a58,3d63345e,2c4e6db0,dcaede90,e720d930,3393b68b,9b23e8ee,a1bbe385,c2baf0bd,cb6010f8,7961e599,d122731b,2dc29e21,222660ff,363d3849,db02c830,d38c2fd7,731c0cbe,15ba1109,58a0de5c,5f5b2617,f71c4741,363c86c9,ac92046e,19967db1,5859dfb6,5e109ec3,26fd2d99,02a42007,daac11b0,63f13dd7,b7530680,99abe2bb,9e34ea74,3bb91dda,51102b85,df4940d3,88d4a5be,49ed92e9,06372577,9ee1c98c,36fa3ebe,bcceccc6,9b4001e4,155f62a4,0330ab6a,1cf54632,7372e1a5,b88f38da,c74f40cd,9ed8f6da,0d1da71f,fd20ea40,bc8f2793,d45ed6a1,bd701df8,895865f3,5a848010,a1192f43,a16a373e,2dcad279,26a5a3dd,d06f75b5,30df3273,160654fd,6cf7d25c,ca11f653,beb0a7b9,28520915,4e5fc6f5,736f9581,b7dc8128,d02b7a8e,29a42aea,56817e2b,1beb320a,562cec5f,756e5507,3dcdda7f,9c5ef70c,dcb55a27,ecc36b7f,2a444e03,93edfe2e,acf5c23f,90ea0bac,d9c005dd,30614231,f50fc6c1,9d4e7b25,6c930e6e,a1e4395d,ad2fc29c,00c73085,77261ab5,37ee8496,e7e44842,15a43e5b,461eace6,77ead60d,55115cbd,8d748b58,cfbd47c8,16667cc5,6077cc36,2b9272f4,47f43a44,15f99afc,d51b1749,2b058fe3,5348fd84,b012cd7f,65abac75,070a5291,e64e2cfd,832735e1,c1cac9a2,4901243f,3d8c61b0,c6971acf,7f0836bf,a29c5338,c0415e5c,3d0b9317,d88ca108,e080a381,5de79a6a,923afab1,7dfe6d8a,1f19558b,4b5efe37,1575e76c,83c6c409,709b1251,2ec694de,7ec0c298,3babcb9b,7525289a,1cc7cfca,3bf1cf26,46b50ba8,e04fb33d,fcfdffb6,85de926c,ecaab346,e5734469,0d18d96c,56cd3b43,df4fe8b6,9b01374f,d88e8f25,0a08139c,499edb7c,44cb4907,2230fab4,15eb4a7d,7cf1bc53,29bdd9ba,51311d7a,4ef8cdd3,f5b8c21a,ad148f58,a8876db3,f6947f54,b80e5e84,7fd1ac25,4d6737eb,3bb91ced,7ad3efc6,4a4c3d21,565a3990,8f094001,a5e9da97,ec138c1c,a0faea5d,c277e121,31973d56,b74258a0,9de5e594,a8efe47b,d3f1e122,0413e89d,de26c3a6,e694a35b,5e812b27,c7128948,d2e9262e,7d093bf9,7040c096,28a4eb9a,01ca3a3c,cdd22e43,bb3e370b,b120f2ac,792530f8,1996c610,77c76bc5,6f445b57,1375ccb7,8d84fa81,f54238ee,3ccd3f02,5b49460a,804ee27f,c54cf6c5,828e68f9,65a38bf7,ecc6157f,0086365d,4d911100,67aa2ada,a76029ee,9ce586dd,c952eb01,84538528,532a2afb,262136f4,392e14df,c7fe2a55,6c517a88,2fb91ec1,bbfe0445,022b4259,3afb49e6,1b54d27f,93b353f2,3323d7e9,16dffff1,37c53127,90d848e0,3a4be871,74e5f8a7,e7561dd2,f32856e4,6aeafed4,33505eae,611485c5,1bb5fbdb,6f8106d9,91561152,3ee399c3,3dfd4aa4,5d042115,76babcde,3ddc79c3,5154fc30,3bfd1a65,8af75982,6f4bd64e,b2dba42b,250513af,05ad839b,ea296733,f93fc684,f56e0afc,6088b756,5f0eb72c,8ac7cce4,a2df0760,4c2ec19f,e4d32835,119b5b02,9e6b7fb5,4bb2f698,0db6d71d,6043a2b4,f3cd5473,a8a78786,85d1b0de,b5053438,a5be6304,08fd73f3,a52b92d5,6d90d394,86ba578b,d2659ab4,1c178d24,17113b36,86c924c4,587b5989,9e4c8c7b,fbaf3456,25fa8af4,9d29771f,f28c589a,884228c8,8b757ab8,e4f1efe6,ea321fb1,bfc77bd6,87d743c1,c7f7f0e1,e37a2b78,795e4a37,14de4c5d,2a512369,47efca07,37db1c2f,b2e5b0f1,c51d8688,92687c59,7da34a02,d185d3ea,5c3d2b2f,a44b10dc,47026d5f,37937459,d3640339,f806dc10,9554a50b,5be391b5,38074c54,56bcd38d,0ce40006,e9c52111,d3268efa,abc5811c,3afde5dd,3b2048ee,ab3136ba,cf82af56,28ed704e,8d7e386c,d2278a3b,28f975ea,90efca10,e5c9df6f,c189aaf2,bd612267,7ab78247,27253bdc,89aace00,8fee50e2,1af8be29,a592d54e,46cd75b4,1325467d,48349b14,3edf6747,Mushroom Sorter (Assessment),Ordering Spheres,Chest Sorter (Assessment),Happy Camel,Chicken Balancer (Activity),Rulers,Crystals Rule,Dino Dive,Dino Drink,Tree Top City - Level 2,Bubble Bath,Crystal C

In [25]:
# quadratic weighted kappa
def qwk3(a1, a2, max_rat=3):
    '''
    a1 - ground truth
    a2 - predicted values
    '''
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)
    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))
    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)
    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)
    e = e / a1.shape[0]
    return 1 - o / e

In [32]:
target = 'accuracy_group'
num_splits = 10
params = {
    'learning_rate': 0.007,
    'metric': 'multiclass',
    'objective': 'multiclass',
    'num_classes': 4,
    'feature_fraction': 0.75,
    "bagging_fraction": 0.8,
    "bagging_seed": 42
}

early_stopping_rounds = 100
num_boost_round = 2000

def train_model(comp_train_df):
    
    kf = KFold(n_splits=num_splits, shuffle=True)
    
    oof_pred = np.zeros((len(comp_train_df), 4))
    models = []
    
    for fold, (tr_ind, val_ind) in enumerate(kf.split(comp_train_df)):
        print(f'Fold: {fold+1}')
        x_train, x_val = comp_train_df[features].iloc[tr_ind], comp_train_df[features].iloc[val_ind]
        y_train, y_val = comp_train_df[target][tr_ind], comp_train_df[target][val_ind]
        train_set = lgb.Dataset(x_train, y_train)
        val_set = lgb.Dataset(x_val, y_val)

        model = lgb.train(params, train_set, num_boost_round = num_boost_round, early_stopping_rounds = early_stopping_rounds, 
                          valid_sets=[train_set, val_set], verbose_eval = early_stopping_rounds)
        oof_pred[val_ind] = model.predict(x_val)
        models.append(model)
        
        val_crt_fold = qwk3(y_val, oof_pred[val_ind].argmax(axis = 1))
        print(f'Fold: {fold+1} quadratic weighted kappa score: {np.round(val_crt_fold,4)}')
        
    res = qwk3(comp_train_df['accuracy_group'], oof_pred.argmax(axis = 1))
    print(f'Quadratic weighted score: {np.round(res,4)}')
        
    return models

In [33]:
models = train_model(comp_train_df)

Fold: 1
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 1.07453	valid_1's multi_logloss: 1.10023
[200]	training's multi_logloss: 0.998016	valid_1's multi_logloss: 1.04563
[300]	training's multi_logloss: 0.948296	valid_1's multi_logloss: 1.01731
[400]	training's multi_logloss: 0.912069	valid_1's multi_logloss: 1.00208
[500]	training's multi_logloss: 0.881584	valid_1's multi_logloss: 0.99295
[600]	training's multi_logloss: 0.855368	valid_1's multi_logloss: 0.98693
[700]	training's multi_logloss: 0.832005	valid_1's multi_logloss: 0.982558
[800]	training's multi_logloss: 0.811039	valid_1's multi_logloss: 0.980049
[900]	training's multi_logloss: 0.792031	valid_1's multi_logloss: 0.978647
[1000]	training's multi_logloss: 0.774002	valid_1's multi_logloss: 0.97795
[1100]	training's multi_logloss: 0.757085	valid_1's multi_logloss: 0.977256
[1200]	training's multi_logloss: 0.741186	valid_1's multi_logloss: 0.977005
[1300]	training's multi_logloss: 0.

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 1.07783	valid_1's multi_logloss: 1.07423
[200]	training's multi_logloss: 1.00187	valid_1's multi_logloss: 1.01523
[300]	training's multi_logloss: 0.952219	valid_1's multi_logloss: 0.982564
[400]	training's multi_logloss: 0.915366	valid_1's multi_logloss: 0.965218
[500]	training's multi_logloss: 0.885238	valid_1's multi_logloss: 0.95509
[600]	training's multi_logloss: 0.859467	valid_1's multi_logloss: 0.947821
[700]	training's multi_logloss: 0.836316	valid_1's multi_logloss: 0.943196
[800]	training's multi_logloss: 0.815253	valid_1's multi_logloss: 0.939747
[900]	training's multi_logloss: 0.795906	valid_1's multi_logloss: 0.938178
[1000]	training's multi_logloss: 0.77762	valid_1's multi_logloss: 0.937279
[1100]	training's multi_logloss: 0.760522	valid_1's multi_logloss: 0.93707
Early stopping, best iteration is:
[1079]	training's multi_logloss: 0.763977	valid_1's multi_logloss: 0.936941
Fold: 8

## Inference

In [ ]:
def run_predictions(models, df):
    y_pred = np.zeros((len(df), 4))
    for i, model in enumerate(models):
        y_pred += model.predict(df[features])
        print(f'Ran {i}th model ')
    return y_pred / num_splits

In [ ]:
y_pred = run_predictions(models[:], comp_test_df)

In [36]:
np.unique(y_pred.argmax(-1), return_counts=True)

(array([0, 1, 2, 3]), array([207,  34,   1, 758]))

In [38]:
comp_test_df['accuracy_group']

array([0])